In [3]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime as dt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from tensorflow.keras.layers import Dense, Input, Dropout, Rescaling, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings('ignore')

2024-10-06 09:12:12.292212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 09:12:12.305405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 09:12:12.308955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 09:12:12.320155: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-06 09:12:13.217665: W tensorflow/compiler/tf2

In [2]:
# Check if GPUs are available for training 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1728198737.663221   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198738.138820   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198738.139078   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
BATCH_SIZE=32

#### Train/test sets

In [4]:
# Train set : CIFAKE/train
# Class FAKE : CIFAKE/train/FAKE
# Class REAL : CIFAKE/train/REAL
train_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/train', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

# Test set : CIFAKE > test
# Class FAKE : CIFAKE/test/FAKE
# Class REAL : CIFAKE/test/REAL
validation_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/test', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

Found 100000 files belonging to 2 classes.


I0000 00:00:1728198758.629844   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198758.630060   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198758.630214   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198758.729480   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 20000 files belonging to 2 classes.


In [5]:
class_names = train_set.class_names
print(f'Training classes: {class_names}')

class_names = validation_set.class_names
print(f'Validation classes: {class_names}')

Training classes: ['FAKE', 'REAL']
Validation classes: ['FAKE', 'REAL']


#### Image normalisation

In [6]:
# Image normalisation

#### Build model

In [66]:
# Build model
def CNN_model01():
    model = Sequential([
        Rescaling(1./255),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),

        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile( optimizer=Adam(learning_rate=0.01), 
                  loss=BinaryCrossentropy(),
                  metrics = ['accuracy', Precision(), Recall()],                
                  )

    model.build(input_shape=(None, 32, 32, 3))
    return model

In [67]:
CNN_model01().summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_8 (Rescaling)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 7200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 24)             │       172,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 173,745 (678.69 KB)

 Trainable params: 173,745 (678.69 KB)

 Non-trainable params: 0 (0.00 B)

#### Train model

In [68]:
N_EPOCHS = 20

In [69]:
time_start = dt.now()
print(f'Start training, time: {time_start.time()}')

model = CNN_model01()

history = model.fit(
  train_set,
  validation_data=validation_set,
  epochs=N_EPOCHS,
  verbose=1
)

print(f'Time elapsed: {dt.now() - time_start}')

Start training, time: 10:01:04.772480
Epoch 1/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7375 - loss: 0.5351 - precision_9: 0.7228 - recall_9: 0.7662 - val_accuracy: 0.7843 - val_loss: 0.4689 - val_precision_9: 0.7289 - val_recall_9: 0.9054
Epoch 2/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7970 - loss: 0.4485 - precision_9: 0.7852 - recall_9: 0.8163 - val_accuracy: 0.8105 - val_loss: 0.4319 - val_precision_9: 0.8315 - val_recall_9: 0.7789
Epoch 3/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8093 - loss: 0.4296 - precision_9: 0.7872 - recall_9: 0.8462 - val_accuracy: 0.8127 - val_loss: 0.4117 - val_precision_9: 0.7698 - val_recall_9: 0.8921
Epoch 4/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8149 - loss: 0.4144 - precision_9: 0.7890 - recall_9: 0.8585 - val_accuracy: 0.8159 - val_loss: 0.4150 - val_precision_9: 0.7693 - val_recall_9: 0.9024
Epoch 5/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8279 - loss: 0

In [75]:
X_train, y_train = train_set[]

from sklearn.model_selection import cross_val_score

k_fold_acc = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')

# NOTE - Try with gridsearch just for cross validation

TypeError: Singleton array array(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
      dtype=object) cannot be considered a valid collection.

#### Visualisation

In [71]:
cols = pd.DataFrame(history.history).columns
cols

Index(['accuracy', 'loss', 'precision_9', 'recall_9', 'val_accuracy',
       'val_loss', 'val_precision_9', 'val_recall_9'],
      dtype='object')

In [72]:
hist = history.history

n = 5
train_col = cols[2]
val_col = f'val_{train_col}'

fig = make_subplots(rows=2, cols=2, subplot_titles=("Loss over epochs", f"{train_col} over epochs"), vertical_spacing=0.07)

# Loss
fig.add_trace( go.Scatter(x=list(range(len(hist['loss']))), y=hist['loss'], mode='lines+markers', name='Training Loss'), row=1, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist['val_loss']))), y=hist['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=1 )

# 
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=1, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=1, col=2)

#
train_col = cols[3]
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=1)

train_col = 'accuracy'
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=2)


fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=1, col=2)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=2)

fig.update_layout(
    # title_text="Training and validation metrics over epochs",
    showlegend=True,
    margin=dict(l=10, r=10, b=10, t=30),
    width=1400, height=800
)

for annotation in fig['layout']['annotations']:
    annotation['y'] = annotation['y'] + 0.002

fig

In [73]:
# Confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred_classes)

#### Class activation map